In [1]:
import numpy as np
import pandas as pd
from random import sample

from coremdlr.config import defaults
from coremdlr.datasets.utils import infer_test_wells

from coremdlr.models import FeaturePredictor, LambdaModel
from coremdlr.datasets import WellLoader, FaciesDataset

Using TensorFlow backend.


In [2]:
import hyperopt
from hyperopt import hp
from hyperopt.pyll.base import scope
from sklearn.metrics import f1_score, log_loss
from sklearn.utils.class_weight import compute_sample_weight

In [3]:
fmodel_args = {
    'logs': {
        'model': 'LambdaModel',
        'model_args': {'feature': 'logs'}
    }
}

XGB_SEARCH_SPACE = {
    'model_type' : 'XGB',
    'max_depth' : scope.int(hp.quniform('max_depth', 3, 10, 1)),
    'learning_rate' : hp.uniform('learning_rate', 0.01, 0.2),
    'n_estimators' : scope.int(hp.quniform('n_estimators', 10, 1000, 1)),
    'objective' : 'multi:softprob',
    'n_jobs' : 2,
    'gamma' : hp.uniform('gamma', 0, 0.5),
    'subsample' : hp.uniform('subsample', 0.3, 1),
    'colsample_bytree' : hp.uniform('colsample_bytree', 0.3, 1.0),
    'colsample_bylevel' : 1,
    'reg_alpha' : 0,                                    # L1 penalty
    'reg_lambda' : hp.uniform('reg_lambda', 0.1, 10),   # L2 penalty
    'tree_method' : 'gpu_exact',
}

def make_params_usable(params):
    """
    Convert params returned by hyperopt to usable `model_args`
    """
    return {
        **XGB_SEARCH_SPACE,
        **params,
        **{'max_depth' : int(params['max_depth']),
           'n_estimators' : int(params['n_estimators'])}
    }

In [4]:
def run_experiment(num_train, max_evals=10, search_space=XGB_SEARCH_SPACE):
    
    assert num_train in [3, 6, 9], 'Must use one of predefined train/test splits'
    
    train_sets = getattr(defaults, f'TRAIN_SETS_{num_train}')
    
    datasets = [FaciesDataset(s, infer_test_wells(s), features=['logs']) for s in train_sets]
    
    for dset in datasets:
        dset.load_or_generate_data()
    
    def train_model(model_config):
        
        log_losses = []
        
        for dset in datasets:
            
            model = FeaturePredictor(dset, model_args=model_config, feature_model_args=fmodel_args)
            model.fit(dset, verbose=False)
            print('F1 score:', f1_score(dset.y_test, model.predict(dset.X_test), average='macro'))
            
            log_losses.append(log_loss(dset.y_test, model.predict_proba(dset.X_test), 
                                      sample_weight=compute_sample_weight('balanced', dset.y_test)))
            
        return np.mean(log_losses)
    
    return hyperopt.fmin(
        fn=train_model,
        space=XGB_SEARCH_SPACE,
        algo=hyperopt.rand.suggest,
        max_evals=max_evals
    )

In [5]:
best_params_3 = run_experiment(3)

print('\n\n\t Starting TRAIN_SET_6')
best_params_6 = run_experiment(6)

print('\n\n\t Starting TRAIN_SET_9')
best_params_9 = run_experiment(9)

Loading Well:  204-24a-7  from  /home/administrator/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  SP
Adding NaN log:  DTS
Feature shapes:  [('depth', (19294,)), ('top', (19294,)), ('base', (19294,)), ('logs', (19294, 11))]
Loading Well:  204-24a-6  from  /home/administrator/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  SP
Adding NaN log:  DTS1
Adding NaN log:  DTS2
Feature shapes:  [('depth', (13006,)), ('top', (13006,)), ('base', (13006,)), ('logs', (13006, 11))]
Loading Well:  204-20-3  from  /home/administrator/Dropbox/core_data/facies/train_data


...

Adding NaN log:  SP
Adding NaN log:  DTS
Feature shapes:  [('depth', (3842,)), ('top', (3842,)), ('base', (3842,)), ('logs', (3842, 11))]
Loading Well:  204-19-7  from  /home/administrator/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  DTS
Feature shapes:  [('depth', (9807,)), ('top', (9807,)), ('base', (9807,)), ('logs', (9807, 11))]
Loading Well:  204-19-3A  from  /home/administrator/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  DTS
Feature shapes:  [('depth', (9590,)), ('top', (9590,)), ('base', (9590,)), ('logs', (9590, 11))]
Training model for feature:  logs
F1 score: 0.18636569176058576
Training model for feature:  logs
F1 score: 0.2516913313195165
Training model for feature:  logs
F1 score: 0.2637556577589334
Training model for feature:  logs
F1 score: 0.17921327596247155
Training model for feature:  logs
F1 score: 0.23057303386341213
Training model for feature:  logs
F1 score: 0.28678486606916176
Training model for feature:  logs
F1 score: 0.1899753884720429
Training model for feature:  logs
F1 score: 0.2221019033776177
Training model for feature:  logs
F1 score: 0.2802362855633773
Training model for feature:  logs
F1 score: 0.2421078239972012
Training model for feature:  logs
F1 score: 0.2237859226556763
Training model for feature:  logs
F1 score: 0.2866977294514573
Training model for feature:  logs
F1 score: 0.19601754213801384
Training model for feature:  logs
F1 score: 0.203380937488

In [6]:
best_params_3

{'colsample_bytree': 0.7065634376490045,
 'gamma': 0.1763363750539652,
 'learning_rate': 0.011349678407368625,
 'max_depth': 4.0,
 'n_estimators': 254.0,
 'reg_lambda': 2.483929898485301,
 'subsample': 0.47226778996571395}

In [5]:
best_params_3 = {'colsample_bytree': 0.7065634376490045,
 'gamma': 0.1763363750539652,
 'learning_rate': 0.011349678407368625,
 'max_depth': 4.0,
 'n_estimators': 254.0,
 'reg_lambda': 2.483929898485301,
 'subsample': 0.47226778996571395}

In [7]:
best_params_6

{'colsample_bytree': 0.5583072645534399,
 'gamma': 0.024522752259214353,
 'learning_rate': 0.1170402654596448,
 'max_depth': 3.0,
 'n_estimators': 423.0,
 'reg_lambda': 9.9419131784328,
 'subsample': 0.9780945495326623}

In [6]:
best_params_6 = {'colsample_bytree': 0.5583072645534399,
 'gamma': 0.024522752259214353,
 'learning_rate': 0.1170402654596448,
 'max_depth': 3.0,
 'n_estimators': 423.0,
 'reg_lambda': 9.9419131784328,
 'subsample': 0.9780945495326623}

In [8]:
best_params_9

{'colsample_bytree': 0.7485373188310873,
 'gamma': 0.04657032367296898,
 'learning_rate': 0.019761179594807568,
 'max_depth': 8.0,
 'n_estimators': 83.0,
 'reg_lambda': 1.324865078210936,
 'subsample': 0.4897623720774287}

In [7]:
best_params_9 = {'colsample_bytree': 0.7485373188310873,
 'gamma': 0.04657032367296898,
 'learning_rate': 0.019761179594807568,
 'max_depth': 8.0,
 'n_estimators': 83.0,
 'reg_lambda': 1.324865078210936,
 'subsample': 0.4897623720774287}

In [9]:
def pred_results_df(num_train, model_config):
    
    assert num_train in [3, 6, 9], 'Must use one of predefined train/test splits'
    
    train_sets = getattr(defaults, f'TRAIN_SETS_{num_train}')
    
    datasets = [FaciesDataset(s, infer_test_wells(s), features=['logs']) for s in train_sets]
    
    pred_dfs = []
    
    for i, dset in enumerate(datasets):
        dset.load_or_generate_data()
        
        model = FeaturePredictor(dset, model_args=model_config, feature_model_args=fmodel_args)
        model.fit(dset, verbose=False)
        
        print('predict_proba: ', model.predict_proba(dset.X_test))
        
        for test_well_name in dset.test_well_names:
            df = model.preds_dataframe(test_well_name, logs=dset.logs_args['which_logs'])
            df['well'] = test_well_name
            df['set_number'] = i
            pred_dfs.append(df)
        
    return pd.concat(pred_dfs)

In [10]:
preds_df_3 = pred_results_df(3, make_params_usable(best_params_3))

(preds_df_3['y_true'] == preds_df_3['y_pred']).sum() / preds_df_3.shape[0]

#preds_df_3.to_csv('logs_xgb_results_train_sets3.csv')

Loading Well:  204-24a-7  from  /home/administrator/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  SP
Adding NaN log:  DTS
Feature shapes:  [('depth', (19294,)), ('top', (19294,)), ('base', (19294,)), ('logs', (19294, 11))]
Loading Well:  204-24a-6  from  /home/administrator/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  SP
Adding NaN log:  DTS1
Adding NaN log:  DTS2
Feature shapes:  [('depth', (13006,)), ('top', (13006,)), ('base', (13006,)), ('logs', (13006, 11))]
Loading Well:  204-20-3  from  /home/administrator/Dropbox/core_data/facies/train_data


...

Header section Parameter regexp=~P was not found.


Feature shapes:  [('depth', (10906,)), ('top', (10906,)), ('base', (10906,)), ('logs', (10906, 11))]
Loading Well:  204-24a-6  from  /home/administrator/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  SP
Adding NaN log:  DTS1
Adding NaN log:  DTS2
Feature shapes:  [('depth', (13006,)), ('top', (13006,)), ('base', (13006,)), ('logs', (13006, 11))]
Training model for feature:  logs
predict_proba:  [[0.42818904 0.19488557 0.16192977 0.11583325 0.0991624 ]
 [0.42818904 0.19488557 0.16192977 0.11583325 0.0991624 ]
 [0.3704394  0.21456788 0.17828374 0.12753172 0.10917724]
 ...
 [0.05701559 0.03342039 0.0911118  0.533365   0.28508723]
 [0.05701559 0.03342039 0.0911118  0.533365   0.28508723]
 [0.05701559 0.03342039 0.0911118  0.533365   0.28508723]]


0.24797880690737834

In [11]:
preds_df_6 = pred_results_df(6, make_params_usable(best_params_6))

(preds_df_6['y_true'] == preds_df_6['y_pred']).sum() / preds_df_6.shape[0]

#preds_df_6.to_csv('logs_xgb_results_train_sets6.csv')

Loading Well:  204-24a-7  from  /home/administrator/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  SP
Adding NaN log:  DTS
Feature shapes:  [('depth', (19294,)), ('top', (19294,)), ('base', (19294,)), ('logs', (19294, 11))]
Loading Well:  204-24a-6  from  /home/administrator/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  SP
Adding NaN log:  DTS1
Adding NaN log:  DTS2
Feature shapes:  [('depth', (13006,)), ('top', (13006,)), ('base', (13006,)), ('logs', (13006, 11))]
Loading Well:  204-20-3  from  /home/administrator/Dropbox/core_data/facies/train_data


...

Header section Parameter regexp=~P was not found.


Adding NaN log:  DTS
Feature shapes:  [('depth', (9590,)), ('top', (9590,)), ('base', (9590,)), ('logs', (9590, 11))]
Loading Well:  204-24a-6  from  /home/administrator/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  SP
Adding NaN log:  DTS1
Adding NaN log:  DTS2
Feature shapes:  [('depth', (13006,)), ('top', (13006,)), ('base', (13006,)), ('logs', (13006, 11))]
Training model for feature:  logs
predict_proba:  [[0.01155851 0.0222644  0.0090724  0.45987186 0.49723282]
 [0.01284787 0.0181854  0.0095796  0.33676535 0.6226218 ]
 [0.0159134  0.02505132 0.01024407 0.4313156  0.5174756 ]
 ...
 [0.05753737 0.11624275 0.50355023 0.2881516  0.03451799]
 [0.05753737 0.11624275 0.50355023 0.2881516  0.03451799]
 [0.09801506 0.11125027 0.48192337 0.27577582 0.03303549]]


0.262596621152511

In [12]:
preds_df_9 = pred_results_df(9, make_params_usable(best_params_9))

(preds_df_9['y_true'] == preds_df_9['y_pred']).sum() / preds_df_9.shape[0]

#preds_df_9.to_csv('logs_xgb_results_train_sets9.csv')

Loading Well:  204-19-3A  from  /home/administrator/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  DTS
Feature shapes:  [('depth', (9590,)), ('top', (9590,)), ('base', (9590,)), ('logs', (9590, 11))]
Loading Well:  204-20-1  from  /home/administrator/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  DTS1
Adding NaN log:  DTS2
Feature shapes:  [('depth', (11210,)), ('top', (11210,)), ('base', (11210,)), ('logs', (11210, 11))]
Loading Well:  204-19-7  from  /home/administrator/Dropbox/core_data/facies/train_data


...

Header section Parameter regexp=~P was not found.


Adding NaN log:  DTS
Feature shapes:  [('depth', (9807,)), ('top', (9807,)), ('base', (9807,)), ('logs', (9807, 11))]
Loading Well:  204-19-3A  from  /home/administrator/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  DTS
Feature shapes:  [('depth', (9590,)), ('top', (9590,)), ('base', (9590,)), ('logs', (9590, 11))]
Training model for feature:  logs
predict_proba:  [[0.4251121  0.05891918 0.29854244 0.07868216 0.1387441 ]
 [0.4251121  0.05891918 0.29854244 0.07868216 0.1387441 ]
 [0.413671   0.06009175 0.30448386 0.08024805 0.14150532]
 ...
 [0.35932723 0.12743595 0.13084775 0.129166   0.2532231 ]
 [0.37275025 0.12476467 0.12810493 0.1264651  0.24791504]
 [0.39381602 0.12057453 0.12380262 0.12221786 0.23958899]]


0.26083007266629404

In [12]:
preds_df_3.shape, preds_df_6.shape, preds_df_9.shape

((152880, 25), (108025, 25), (71560, 25))

In [13]:
preds_df_3.to_csv('logs_xgb_results_train_sets3.csv')
preds_df_6.to_csv('logs_xgb_results_train_sets6.csv')
preds_df_9.to_csv('logs_xgb_results_train_sets9.csv')